In [1]:
import pandas as pd

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import time

import re

In [3]:
from bs4 import BeautifulSoup

In [5]:
# 웹드라이버 접속
driver = webdriver.Chrome(executable_path=r'C:\Users\jennie\Desktop\BOAZ\chromedriver_win32\chromedriver.exe')

In [4]:
# 접속할 url 불러오기
df = pd.read_csv('노원_url수집된것만.csv'); df.head()

,Unnamed: 0,Unnamed: 0.1,상호명,상권업종대분류명,표준산업분류명,시군구명,법정동명,도로명주소,위도,경도,검색어,역 명,거리,naver_keyword,naver_map_url
0,0,8,미가족발,음식,한식 음식점업,노원구,공릉동,서울특별시 노원구 동일로193길 19,37.625793,127.071582,공릉동 미가족발,7호선 공릉(서울과학기술대),110.998565,공릉동%20미가족발,https://pcmap.place.naver.com/restaurant/38323...
1,1,10,파란파랑,음식,한식 음식점업,노원구,상계동,서울특별시 노원구 동일로217길 58,37.653247,127.057902,상계동 파란파랑,7호선 노원,254.794028,상계동%20파란파랑,https://pcmap.place.naver.com/restaurant/36215...
2,2,11,전주백반,음식,한식 음식점업,노원구,공릉동,서울특별시 노원구 동일로173가길 101,37.622752,127.072118,공릉동 전주백반,7호선 공릉(서울과학기술대),351.645144,공릉동%20전주백반,https://pcmap.place.naver.com/restaurant/20919...
3,3,26,와우곱창,음식,한식 음식점업,노원구,공릉동,서울특별시 노원구 공릉로 107,37.619985,127.077749,공릉동 와우곱창,7호선 태릉입구,251.418936,공릉동%20와우곱창,https://pcmap.place.naver.com/restaurant/20877...
4,4,28,어촌,음식,일식 음식점업,노원구,월계동,서울특별시 노원구 석계로8가길 29,37.615406,127.064904,월계동 어촌,1호선 석계,65.926456,월계동%20어촌,https://pcmap.place.naver.com/restaurant/10483...


In [21]:
# 수집할 정보들
review_list = []

In [22]:
for i in range(len(df)):
    
    print('======================================================')
    print('노원구 '+str(i)+'번째 식당')
    
    # 식당 리뷰 개별 url 접속
    driver.get(df['naver_map_url'][i])
    
    
    # 더보기 버튼 다 누르기
    # 더보기 버튼 누르기까지 10개
    # 더보기 버튼 누르면 10개 추가됨
    while True:
        try:
            driver.find_element_by_xpath('//*[@id="app-root"]/div/div[2]/div[5]/div[4]/div[4]/div[2]/a').click()
            time.sleep(2)
        except:
            print('-더보기 버튼 모두 클릭 완료-')
            break
    
    # 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    time.sleep(1)
    
    # 식당 구분
    restaurant_name = df['상호명'][i]
    print('식당 이름 : '+df['법정동명'][i]+' '+df['상호명'][i])
    try:
        restaurant_classificaton = soup.find_all('span',attrs = {'class':'_3ocDE'})[0].text
    except:
        restaurant_classificaton = 'none'
    print('식당 구분 : '+restaurant_classificaton)
    print('----------------------------------------------')
    
    try:
        one_review = soup.find_all('div', attrs = {'class':'_1Z_GL'})
        review_num = len(one_review) # 특정 식당의 리뷰 총 개수
        print('리뷰 총 개수 : '+str(review_num))
    
        # 리뷰 개수
        for i in range(len(one_review)):
        
            # user url
            user_url = one_review[i].find('div', attrs = {'class':'_23Rml'}).find('a').get('href')
            print('user_url = '+user_url)
            
            # user url로부터 user code 뽑아내기
            user_code = re.findall(r"my/(\w+)", user_url)[0]
            print('user_code = '+user_code)
            
            # rating, 별점
            rating = one_review[i].find('span', attrs = {'class':'_2tObC'}).text
            print('rating = '+rating)
            
            # n번째 방문
            nth_visit = one_review[i].find_all('span', attrs = {'class':'_3WqoL'})[4].text
            print('n번째 방문 = '+nth_visit)
            
            # date
            date = one_review[i].find_all('span', attrs = {'class':'_3WqoL'})[3].text
            print('date = '+date)
            
            # review 내용
            try : 
                review_content = one_review[i].find('span', attrs = {'class':'WoYOw'}).text
            except: # 리뷰가 없다면
                review_content = "리뷰 내용 없음"
            print('리뷰 내용 : '+review_content)
            print('----------------------------------------------')
            print('\n')
    
            리뷰정보 = restaurant_name, restaurant_classificaton, user_url, user_code, review_num, rating, nth_visit, date, review_content
            review_list.append(리뷰정보)
    
    except NoSuchElementException:
            none_review = "네이버 리뷰 없음"
            print(none_review)
            review_num = 0
            리뷰정보 = restaurant_name, restaurant_classification, review_num, none_review
            review_list.append(리뷰정보)
            
    print('\n')

노원구 0번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 공릉동 미가족발
식당 구분 : 족발,보쌈
----------------------------------------------
리뷰 총 개수 : 0


노원구 1번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 상계동 파란파랑
식당 구분 : 카페,디저트
----------------------------------------------
리뷰 총 개수 : 10
user_url = https://m.place.naver.com/my/5fb281bfb61cc4c3dd9bcc20/review
user_code = 5fb281bfb61cc4c3dd9bcc20
rating = 4.5
n번째 방문 = 영수증
date = 1번째 방문
리뷰 내용 : 블루베리 음료 맛나요
----------------------------------------------


user_url = https://m.place.naver.com/my/5e1c524b8f87a842bc580991/review
user_code = 5e1c524b8f87a842bc580991
rating = 5
n번째 방문 = 1번째 방문
date = 2021.01.27
리뷰 내용 : 좋아요
----------------------------------------------


user_url = https://m.place.naver.com/my/5e2afda88f87a842bc86c828/review
user_code = 5e2afda88f87a842bc86c828
rating = 5
n번째 방문 = 2번째 방문
date = 2021.01.24
리뷰 내용 : 커피맛나요~
----------------------------------------------


user_url = https://m.place.naver.com/my/5e2afda88f87a842bc86c828/review
user_code = 5e2afda88f87a842bc86c82

-더보기 버튼 모두 클릭 완료-
식당 이름 : 월계동 탑스파
식당 구분 : 목욕탕,사우나
----------------------------------------------
리뷰 총 개수 : 8
user_url = https://m.place.naver.com/my/5e903e8b8f87a842bccff9fc/review
user_code = 5e903e8b8f87a842bccff9fc
rating = 4
n번째 방문 = 영수증
date = 1번째 방문
리뷰 내용 : 넖고 좋아요
----------------------------------------------


user_url = https://m.place.naver.com/my/5f0d45019ec8258e4a2499ea/review
user_code = 5f0d45019ec8258e4a2499ea
rating = 4
n번째 방문 = 영수증
date = 2번째 방문
리뷰 내용 : 좋아요
----------------------------------------------


user_url = https://m.place.naver.com/my/5cb6e56bfcb8ffc33ee5070c/review
user_code = 5cb6e56bfcb8ffc33ee5070c
rating = 4.5
n번째 방문 = 1번째 방문
date = 2020.10.18
리뷰 내용 : 자주 가는데 편하고 좋아요~
----------------------------------------------


user_url = https://m.place.naver.com/my/5f0d45019ec8258e4a2499ea/review
user_code = 5f0d45019ec8258e4a2499ea
rating = 5
n번째 방문 = 영수증
date = 1번째 방문
리뷰 내용 : 좋아요
----------------------------------------------


user_url = https://m.place.naver.co

KeyboardInterrupt: 

In [16]:
driver = webdriver.Chrome(executable_path=r'C:\Users\jennie\Desktop\BOAZ\chromedriver_win32\chromedriver.exe')

for i in range(830,len(df)):
    
    print('======================================================')
    print('노원구 '+str(i)+'번째 식당')
    
    # 식당 리뷰 개별 url 접속
    driver.get(df['naver_map_url'][i])
    
    
    # 더보기 버튼 다 누르기
    # 더보기 버튼 누르기까지 10개
    # 더보기 버튼 누르면 10개 추가됨
    while True:
        try:
            driver.find_element_by_xpath('//*[@id="app-root"]/div/div[2]/div[5]/div[4]/div[4]/div[2]/a').click()
            time.sleep(2)
        except:
            print('-더보기 버튼 모두 클릭 완료-')
            break
    
    # 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    time.sleep(1)
    
    # 식당 구분
    restaurant_name = df['상호명'][i]
    print('식당 이름 : '+df['법정동명'][i]+' '+df['상호명'][i])
    try:
        restaurant_classificaton = soup.find_all('span',attrs = {'class':'_3ocDE'})[0].text
    except:
        restaurant_classificaton = 'none'
    print('식당 구분 : '+restaurant_classificaton)
    print('----------------------------------------------')
    
    try : 
        one_review = soup.find_all('div', attrs = {'class':'_1Z_GL'})
        review_num = len(one_review) # 특정 식당의 리뷰 총 개수
        print('리뷰 총 개수 : '+str(review_num))
    
        # 리뷰 개수
        for i in range(len(one_review)):
        
            # user url
            user_url = one_review[i].find('div', attrs = {'class':'_23Rml'}).find('a').get('href')
            print('user_url = '+user_url)
            
            # user url로부터 user code 뽑아내기
            user_code = re.findall(r"my/(\w+)", user_url)[0]
            print('user_code = '+user_code)
            
            # rating, 별점
            rating = one_review[i].find('span', attrs = {'class':'_2tObC'}).text
            print('rating = '+rating)
            
            # n번째 방문
            nth_visit = one_review[i].find_all('span', attrs = {'class':'_3WqoL'})[4].text
            print('n번째 방문 = '+nth_visit)
            
            # date
            date = one_review[i].find_all('span', attrs = {'class':'_3WqoL'})[3].text
            print('date = '+date)
            
            # review 내용
            try : 
                review_content = one_review[i].find('span', attrs = {'class':'WoYOw'}).text
            except: # 리뷰가 없다면
                review_content = "리뷰 내용 없음"
            print('리뷰 내용 : '+review_content)
            print('----------------------------------------------')
            print('\n')
    
            리뷰정보 = restaurant_name, restaurant_classificaton, user_url, user_code, review_num, rating, nth_visit, date, review_content
            review_list.append(리뷰정보)
    
    except NoSuchElementException:
            none_review = "네이버 리뷰 없음"
            print(none_review)
            review_num = 0
            리뷰정보 = restaurant_name, restaurant_classification, review_num, none_review
            review_list.append(리뷰정보)
            
    print('\n')

노원구 830번째 식당
-더보기 버튼 모두 클릭 완료-
식당 이름 : 상계동 밀알생돈까스
식당 구분 : 일식당
----------------------------------------------
리뷰 총 개수 : 10
user_url = https://m.place.naver.com/my/5eff440d598cfed34e17978b/review
user_code = 5eff440d598cfed34e17978b
rating = 3
n번째 방문 = 1번째 방문
date = 2021.03.19
리뷰 내용 : 사장님께서 청소에 열정적이세요. 굉장히 깔끔.샐러드도 신선하고 좋아요. 김치나베돈까스먹었는데 고기가 꽤 질긴 거 빼곤 괜찮았어요.
----------------------------------------------


user_url = https://m.place.naver.com/my/5c72b6f3f8e62ec41dfdc251/review
user_code = 5c72b6f3f8e62ec41dfdc251
rating = 4.5
n번째 방문 = 영수증
date = 2번째 방문
리뷰 내용 : 조아요
----------------------------------------------


user_url = https://m.place.naver.com/my/5f216b909ec8258e4a73e2fc/review
user_code = 5f216b909ec8258e4a73e2fc
rating = 4
n번째 방문 = 영수증
date = 3번째 방문
리뷰 내용 : 좋아요
----------------------------------------------


user_url = https://m.place.naver.com/my/5b9d21216b934be114fac35a/review
user_code = 5b9d21216b934be114fac35a
rating = 4
n번째 방문 = 1번째 방문
date = 2021.03.11
리뷰 내용 : 맛있는데 반찬이 셀프였음 

KeyboardInterrupt: 

In [23]:
df_new = pd.DataFrame(review_list, columns = ['restaurant_name', 'restaurant_classificaton', 'user_url', 'user_code', 'reveiw_num', 'rating', 'nth_visit', 'date', 'review_content'])
df_new

,restaurant_name,restaurant_classificaton,user_url,user_code,reveiw_num,rating,nth_visit,date,review_content
0,파란파랑,"카페,디저트",https://m.place.naver.com/my/5fb281bfb61cc4c3d...,5fb281bfb61cc4c3dd9bcc20,10,4.5,영수증,1번째 방문,블루베리 음료 맛나요
1,파란파랑,"카페,디저트",https://m.place.naver.com/my/5e1c524b8f87a842b...,5e1c524b8f87a842bc580991,10,5,1번째 방문,2021.01.27,좋아요
2,파란파랑,"카페,디저트",https://m.place.naver.com/my/5e2afda88f87a842b...,5e2afda88f87a842bc86c828,10,5,2번째 방문,2021.01.24,커피맛나요~
3,파란파랑,"카페,디저트",https://m.place.naver.com/my/5e2afda88f87a842b...,5e2afda88f87a842bc86c828,10,5,1번째 방문,2020.11.02,집앞이라 일주일에 두세번은 가는거 같아요 맛있어서 가는거에요!
4,파란파랑,"카페,디저트",https://m.place.naver.com/my/5c74c9695e722ae41...,5c74c9695e722ae418d30dd9,10,5,1번째 방문,2020.10.06,맛있어요
5,파란파랑,"카페,디저트",https://m.place.naver.com/my/5ee80a238f87a842b...,5ee80a238f87a842bc994c6b,10,4,영수증,3번째 방문,굿
6,파란파랑,"카페,디저트",https://m.place.naver.com/my/5e3ccabb8f87a842b...,5e3ccabb8f87a842bc8ffb72,10,4,3번째 방문,2020.09.29,굿
7,파란파랑,"카페,디저트",https://m.place.naver.com/my/5ee80a238f87a842b...,5ee80a238f87a842bc994c6b,10,4,영수증,2번째 방문,굿
8,파란파랑,"카페,디저트",https://m.place.naver.com/my/5e3ccabb8f87a842b...,5e3ccabb8f87a842bc8ffb72,10,4,2번째 방문,2020.09.19,굿
9,파란파랑,"카페,디저트",https://m.place.naver.com/my/5ee80a238f87a842b...,5ee80a238f87a842bc994c6b,10,3.5,영수증,1번째 방문,굿


In [59]:
df_new.to_csv('노원리뷰.csv', encoding = 'utf-8-sig')